In [1]:
import pandas as pd
import numpy as np
import re  
from lib.config import csconnstr
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender


In [2]:
input="""
Cyber Security Support
Financial Information Support
Medical/health care Support
Data storage and management
Legal Services
Law enforcement services
Veteran support services
Immigration services
Data Access and Sharing
NASA and space services
Access control
Visitor management
Video surveillance
PII data control
Communication
Emergency dispatch
IT Support service
Data center/warehouse
Emergency Alerts
Real Time data access
Physical Security
Weather Display and alerts
Payroll related Services
Web Based Monitoring
Provides infrastructure services
Other [Free Text]

""".split('\n')

pr = picklist_recommender(csconnstr)
r=pr.recommend(input, threshhold=.8, usecosine_sim=False) 
print(r)
print(pr.get_script())

{'PK_PicklistType': 0, 'UsageField': 0, 'DisplayValue': 'CYBER SECURITY SUPPORTFINANCIAL INFORMATION SUPPORTMEDICAL HEALTH CARE SUPPORTDATA STORAGE AND MANAGEMENTLEGAL SERVICESLAW ENFORCEMENT SERVICESVETERAN SUPPORT SERVICESIMMIGRATION SERVICESDATA ACCESS AND SHARINGNASA AND SPACE SERVICESACCESS CONTROLVISITOR MANAGEMENTVIDEO SURVEILLANCEPII DATA CONTROLCOMMUNICATIONEMERGENCY DISPATCHIT SUPPORT SERVICEDATA CENTER WAREHOUSEEMERGENCY ALERTSREAL TIME DATA ACCESSPHYSICAL SECURITYWEATHER DISPLAY AND ALERTSPAYROLL RELATED SERVICESWEB BASED MONITORINGPROVIDES INFRASTRUCTURE SERVICESOTHER FREE TEXT', 'PK_Picklist': 4185, 'MAX_PK_Picklist': 10172, 'MAX_PK_PicklistType': 387, 'agg_simscore': 0.3178082191780822, 'sequence_match_score': 0.3178082191780822}
(PK_PickList=10182, PK_PicklistType=389, SortOrder=1, UsageCode='CYBER_SECUR_SUP', DisplayValue='Cyber Security Support')
,(PK_PickList=10183, PK_PicklistType=389, SortOrder=2, UsageCode='FINANCI_INPPORT', DisplayValue='Financial Information Sup

In [16]:
df = sql_todf("""
SELECT PK_Picklist, PK_PicklistType, UsageField, DisplayValue from vwPicklists
""", csconnstr)  
df=df.loc[len(df)-1, ['PK_Picklist','PK_PicklistType']]
df

PK_Picklist        10172
PK_PicklistType      387
Name: 3069, dtype: object

In [4]:
def tokenize(s): 
    s = [v for v in s]  
    s = re.sub('[^A-Za-z0-9\s]','', ' '.join(s)).split() 
    return s

In [85]:
def normalize(s):  
    s = re.sub('[^A-Za-z0-9\s]',' ',s).upper().strip()
    s = re.sub('\s{1,5}',' ',s) 
    return s
normalize('   aASDF    ASDF   QWER QWER    QWERQWER')

'AASDF ASDF QWER QWER QWERQWER'

In [47]:
def jaccard_similarity(x,y): 
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [145]:

for i,r in df[167:168].iterrows():
    print(r['PK_PicklistType']) 
    x=normalize(r['DisplayValue'])
    print(x) 


294
NOT APPLICABLE ACQUIA ADOBE AT T AKAMAI AMAZON BLACKBERRY BOX CGI CISCO SYSTEMS EVERBRIDGE GOOGLE GRANICUS IBM LUMEN TECHNOLOGIES MICROSOFT MICROPACT ORACLE SAP SERVICENOW SPRINT VERIZON OTHER SALESFORCE


NameError: name 'picklist_recommender' is not defined

In [187]:
def recommend_picklist(input_list, threshhold=.8, usecosine_sim=False):
    df = sql_todf("SELECT PK_PicklistType, UsageField, DisplayValue from vwPicklists  ", csconnstr)  
    df = df.groupby(['PK_PicklistType','UsageField'], as_index = False).agg({'DisplayValue': ' '.join})       
    input=normalize(''.join(input_list))
    for i,r in df.iterrows(): 
        x=normalize(r['DisplayValue'])
        sequence_match = SequenceMatcher(None, x,input).ratio() 
        df.at[i,'sequence_match']= sequence_match   
        df.at[i,'cosine_sim'] = 0
        if df.at[i,'sequence_match'] > threshhold: 
            return df.iloc[i].to_dict()    
        if usecosine_sim:        
            vectorizer = TfidfVectorizer()
            X = vectorizer.fit_transform([x,input])  
            cosine_sim = cosine_similarity(X, X)  
            df.at[i,'cosine_sim']= cosine_sim[0][1]  
            if df.at[i,'cosine_sim'] > threshhold:
                return df.iloc[i].to_dict()
    df=df.sort_values(by=['cosine_sim', 'sequence_match'], ascending=[False,False], inplace=False)
    d=df.iloc[0].to_dict()    
    return d

input="""
1. Security Operations Center (SOC) as a Service such as Asset Management
2. Awareness and Training
3. Detection Processes
4. Governance: Information Protection Processes and Procedures;
5. Protective Technology
6. Risk Assessment
7. Security Continuous Monitoring
8. Supply Chain Risk Management

""".split('\n') 
recommend_picklist(input)

{'PK_PicklistType': 101,
 'UsageField': 'SAOP2',
 'DisplayValue': 'Policy making Compliance Risk management activities',
 'sequence_match': 0.1276595744680851,
 'cosine_sim': 0.0}

In [48]:
l=df.to_dict('records')
for d in l[:5]:
    print(d)
x = normalize(df['DisplayValue'].values)
x

['LEGACY CODEBASE COBOL MUMPS',
 'LACK OF OR POOR DOCUMENTATION',
 'LEGACY INFRASTRUCTURE',
 'AD HOC INFRASTRUCTURE',
 'HARDWARE OR SOFTWARE WITH NO VENDOR MANUFACTURER SUPPORT',
 'OUTDATED OR INACCURATE INVENTORY',
 'TECHNOLOGY TYPE',
 'SYSTEM LAST MODERNIZED',
 'LIMITED FUNCTIONALITY',
 'OTHER']

In [49]:
y="""Legacy codebase (COBOL MUMPS)
Lack of or poor documentation
Legacy infrastructure
Ad-hoc infrastructure 
Hardware or software with no vendor/manufacturer support
Outdated or inaccurate inventory
Limited Functionality
other
""".split('\n')
y= normalize(y)
y


['LEGACY CODEBASE COBOL MUMPS',
 'LACK OF OR POOR DOCUMENTATION',
 'LEGACY INFRASTRUCTURE',
 'AD HOC INFRASTRUCTURE',
 'HARDWARE OR SOFTWARE WITH NO VENDOR MANUFACTURER SUPPORT',
 'OUTDATED OR INACCURATE INVENTORY',
 'LIMITED FUNCTIONALITY',
 'OTHER',
 '']

0.8421052631578947
0.7272727272727273


[[1.         0.84904082]
 [0.84904082 1.        ]]


In [ ]:
threshold = .1
for x in range(0,X.shape[0]):
  for y in range(x,X.shape[0]):
    if(x!=y):
      if(cosine_similarity(X[x],X[y])>threshold):
        print(df["ID"][x],":",corpus[x])
        print(df["ID"][y],":",corpus[y])
        print("Cosine similarity:",cosine_similarity(X[x],X[y]))
        print()

In [51]:
corpus = x 
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
df_idf = pd.DataFrame(vectorizer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

<10x30 sparse matrix of type '<class 'numpy.float64'>'
	with 34 stored elements in Compressed Sparse Row format>

In [63]:
df = pd.DataFrame(columns=["ID","DESCRIPTION"], data=np.matrix([[10,"Cancel ASN WMS Cancel ASN"],
                                                                [11,"MAXPREDO Validation is corect"],
                                                                [12,"Move to QC"],
                                                                [13,"Cancel ASN WMS Cancel ASN"],
                                                                [14,"MAXPREDO Validation is right"],
                                                                [15,"Verify files are sent every hours for this interface from Optima"],
                                                                [16,"MAXPREDO Validation are correct"],
                                                                [17,"Move to QC"],
                                                                [18,"Verify files are not sent"]
                                                                ]))

corpus = list(df["DESCRIPTION"].values)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

threshold = 0.4

for x in range(0,X.shape[0]):
  for y in range(x,X.shape[0]):
    if(x!=y):
      if(cosine_similarity(X[x],X[y])>threshold):
        print(df["ID"][x],":",corpus[x])
        print(df["ID"][y],":",corpus[y])
        print("Cosine similarity:",cosine_similarity(X[x],X[y]))
        print()

10 : Cancel ASN WMS Cancel ASN
13 : Cancel ASN WMS Cancel ASN
Cosine similarity: [[1.]]

11 : MAXPREDO Validation is corect
14 : MAXPREDO Validation is right
Cosine similarity: [[0.64183024]]

12 : Move to QC
17 : Move to QC
Cosine similarity: [[1.]]

15 : Verify files are sent every hours for this interface from Optima
18 : Verify files are not sent
Cosine similarity: [[0.44897995]]



In [28]:

def sql_todf(query,connstr):
    df=pd.DataFrame() 
    engine = create_engine(connstr) 
    conn = engine.connect() 
    #try: 
    df = pd.read_sql(query,con=conn) 
    #finally: 
    conn.close()
    return df

['LEGACY',
 'CODEBASE',
 '(COBOL',
 'MUMPS)',
 'LACK',
 'OF',
 'OR',
 'POOR',
 'DOCUMENTATION',
 'LEGACY',
 'INFRASTRUCTURE',
 'AD-HOC',
 'INFRASTRUCTURE',
 '',
 'HARDWARE',
 'OR',
 'SOFTWARE',
 'WITH',
 'NO',
 'VENDOR/MANUFACTURER',
 'SUPPORT',
 'OUTDATED',
 'OR',
 'INACCURATE',
 'INVENTORY',
 'TECHNOLOGY',
 'TYPE',
 '',
 'SYSTEM',
 'LAST',
 'MODERNIZED',
 'LIMITED',
 'FUNCTIONALITY',
 'OTHER',
 '']

In [58]:
y="""Legacy codebase (COBOL MUMPS)
Lack of or poor documentation
Legacy infrastructure
Ad-hoc infrastructure 
Hardware or software with no vendor/manufacturer support
Outdated or inaccurate inventory
Limited Functionality
""".replace('\n',' ').upper().split(' ')
y

['LEGACY',
 'CODEBASE',
 '(COBOL',
 'MUMPS)',
 'LACK',
 'OF',
 'OR',
 'POOR',
 'DOCUMENTATION',
 'LEGACY',
 'INFRASTRUCTURE',
 'AD-HOC',
 'INFRASTRUCTURE',
 '',
 'HARDWARE',
 'OR',
 'SOFTWARE',
 'WITH',
 'NO',
 'VENDOR/MANUFACTURER',
 'SUPPORT',
 'OUTDATED',
 'OR',
 'INACCURATE',
 'INVENTORY',
 'LIMITED',
 'FUNCTIONALITY',
 '']

In [59]:
x=lst

jaccard_similarity(x,y)

0.7931034482758621